In [1]:
import pickle
import gzip
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/X_train.pickle','rb') as f:
    data = pickle.load(f)

In [8]:
x_train= data
del data

In [10]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/X_val.pickle','rb') as f:
    x_val = pickle.load(f)

In [15]:
with gzip.open('/content/drive/MyDrive/Colab Notebooks/ked_Data/Y_train.pickle','rb') as f:
    y_train = pickle.load(f)

In [11]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/Y_val.pickle','rb') as f:
    y_val = pickle.load(f)

In [ ]:
#with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/X_test.pickle','rb') as f:
  #  x_test = pickle.load(f)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
tf.random.set_seed(777)

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [12]:
threshold = 11
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 53868
등장 빈도가 10번 이하인 희귀 단어의 수: 35211
단어 집합에서 희귀 단어의 비율: 65.36533749164624
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.2151456716624007


In [13]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 18658


In [14]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(x_train)
token_X_train = tokenizer.texts_to_sequences(x_train)
token_X_val = tokenizer.texts_to_sequences(x_val)

In [16]:
drop_train = [index for index, sentence in enumerate(token_X_train) if len(sentence) < 1]

In [17]:
print(type(y_train))
print(type(drop_train))

<class 'pandas.core.series.Series'>
<class 'list'>


In [18]:
y_train

911502     G
1381888    F
1437301    I
172971     M
598706     E
          ..
259178     F
1414414    I
131932     G
671155     C
121958     E
Name: Big, Length: 1139568, dtype: object

In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
import numpy as np
del_X_train = np.delete(token_X_train, drop_train, axis=0)
del_y_train = np.delete(y_train, drop_train, axis=0)
print(len(del_X_train))
print(len(del_y_train))

1139566
1139566


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [21]:
del_y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigtrain_y.pickle', 'wb') as f:
    pickle.dump(del_y_train, f)